In [1]:
import pandas as pd
from json import load
import urllib.request, json 
from pandas.io.json import json_normalize
import seaborn as sns
import pylab as plt
import multiprocessing as mp
%matplotlib inline

In [2]:
url_root = 'https://mriqc.nimh.nih.gov/api/v1/{modality}?where={query}'
software = 'mriqc'
version = '0.9.6'

page = 1
dfs = []
while True:
    print('Querying page %d' % page)
    page_url = url_root.format(
        modality='bold',
        query='{"provenance.software":"%s","provenance.version":"%s"}&page=%d' % (software, version, page)
    )
    with urllib.request.urlopen(page_url) as url:
        data = json.loads(url.read().decode())
        dfs.append(json_normalize(data['_items']))
        if 'next' not in data['_links'].keys():
            break
        else:
            page += 1

# Compose a pandas dataframe
df = pd.concat(dfs, ignore_index=True)

Querying page 1
Querying page 2
Querying page 3
Querying page 4
Querying page 5
Querying page 6
Querying page 7
Querying page 8
Querying page 9
Querying page 10
Querying page 11
Querying page 12
Querying page 13
Querying page 14
Querying page 15
Querying page 16
Querying page 17
Querying page 18
Querying page 19
Querying page 20
Querying page 21
Querying page 22
Querying page 23
Querying page 24
Querying page 25
Querying page 26
Querying page 27
Querying page 28
Querying page 29
Querying page 30
Querying page 31
Querying page 32
Querying page 33
Querying page 34
Querying page 35
Querying page 36
Querying page 37
Querying page 38
Querying page 39
Querying page 40
Querying page 41
Querying page 42
Querying page 43
Querying page 44
Querying page 45
Querying page 46
Querying page 47
Querying page 48
Querying page 49
Querying page 50
Querying page 51
Querying page 52
Querying page 53
Querying page 54
Querying page 55
Querying page 56
Querying page 57
Querying page 58
Querying page 59
Queryi

In [3]:
df.describe()

,aor,aqi,bids_meta.EchoTime,bids_meta.EchoTrainLength,bids_meta.EffectiveEchoSpacing,bids_meta.FlipAngle,bids_meta.ImagingFrequency,bids_meta.InversionTime,bids_meta.MagneticFieldStrength,bids_meta.MultibandAccelerationFactor,...,summary_bg_stdv,summary_fg_k,summary_fg_mad,summary_fg_mean,summary_fg_median,summary_fg_n,summary_fg_p05,summary_fg_p95,summary_fg_stdv,tsnr
count,9447.000000,9447.000000,8327.000000,144.0,5796.000000,8195.000000,144.0,144.0,5571.000000,239.0,...,9447.000000,9447.000000,9447.000000,9447.000000,9447.000000,9447.000000,9447.000000,9.447000e+03,9447.000000,9447.000000
mean,0.004177,0.008920,0.030753,1.0,0.000481,80.547407,123.0,0.0,2.981332,3.0,...,617.629417,2.603478,1855.872716,10800.092031,10836.871634,52795.961258,7060.231977,1.434668e+04,2262.119393,56.108772
std,0.005515,0.011376,0.030152,0.0,0.000160,12.974047,0.0,0.0,0.192339,0.0,...,4335.969289,1.352919,13539.657050,83994.128198,84593.856746,47536.804804,56090.598904,1.095089e+05,16601.600836,18.785432
min,0.000193,0.000852,0.003500,1.0,0.000042,7.000000,123.0,0.0,1.000000,3.0,...,6.518906,-0.945427,19.791393,134.614075,136.134552,14446.000000,72.217881,1.741673e+02,27.285780,0.195811
25%,0.001210,0.004176,0.030000,1.0,0.000395,78.000000,123.0,0.0,3.000000,3.0,...,46.025280,1.641122,95.714607,588.857361,579.894836,29322.000000,329.948755,8.003506e+02,132.774506,42.585161
50%,0.002423,0.006849,0.030000,1.0,0.000500,80.000000,123.0,0.0,3.000000,3.0,...,62.897224,2.493541,153.675201,724.306152,696.422607,35210.000000,416.626678,1.093099e+03,195.619125,55.794506
75%,0.004818,0.010927,0.030000,1.0,0.000560,90.000000,123.0,0.0,3.000000,3.0,...,83.917118,3.497120,224.253258,925.607697,936.266724,48221.000000,567.550217,1.351433e+03,276.941696,67.819988
max,0.146135,0.875518,0.400000,1.0,0.000730,90.000000,123.0,0.0,3.000000,3.0,...,51126.781250,12.063769,144198.171875,924981.437500,927293.625000,496934.000000,659703.012500,1.183218e+06,175338.796875,127.314201


In [4]:
df_unique.drop_duplicates(subset=['provenance.md5sum'])
df_unique.describe()

NameError: name 'df_unique' is not defined

In [ ]:
sns.distplot(df_unique.dvars_nstd)
plt.xlabel("Mean Framewise Displacement [mm]")